In [1]:
#Libraries
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score

import re,string,unicodedata

import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from bs4 import BeautifulSoup

In [2]:
# nltk.download('stopwords')

In [3]:
df_fake = pd.read_csv('C:/Users/Boonchong/Desktop/Intern/FakeNews/Fake.csv')
df_true = pd.read_csv('C:/Users/Boonchong/Desktop/Intern/FakeNews/True.csv')

In [4]:
df_true['classification'] = 1
df_fake['classification'] = 0

In [5]:
df_fake.head()

,title,text,subject,date,classification
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


#### Data Cleaning




In [6]:
df_true.head()
# Note that most of True texts have (Reuters)
# Some has disclaimer where real tweets come after @realDonaldTrump : -

,title,text,subject,date,classification
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [7]:
df_true.text

0        WASHINGTON (Reuters) - The head of a conservat...
1        WASHINGTON (Reuters) - Transgender people will...
2        WASHINGTON (Reuters) - The special counsel inv...
3        WASHINGTON (Reuters) - Trump campaign adviser ...
4        SEATTLE/WASHINGTON (Reuters) - President Donal...
                               ...                        
21412    BRUSSELS (Reuters) - NATO allies on Tuesday we...
21413    LONDON (Reuters) - LexisNexis, a provider of l...
21414    MINSK (Reuters) - In the shadow of disused Sov...
21415    MOSCOW (Reuters) - Vatican Secretary of State ...
21416    JAKARTA (Reuters) - Indonesia will buy 11 Sukh...
Name: text, Length: 21417, dtype: object

In [8]:
clean = []
for data in df_true.text:
    if "@realDonaldTrump : - " in data:
        clean.append(data.split("@realDonaldTrump : - ")[1])
    elif "(Reuters) -" in data:
        clean.append(data.split("(Reuters) - ")[1])
    else:
        clean.append(data)
        
df_true.text = clean

In [9]:
df_true.head()

,title,text,subject,date,classification
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...,politicsNews,"December 29, 2017",1


In [10]:
df = pd.concat([df_fake,df_true])

In [11]:
df.isna().sum()

title             0
text              0
subject           0
date              0
classification    0
dtype: int64

In [12]:
df[(df.classification == 0)].count()

title             23481
text              23481
subject           23481
date              23481
classification    23481
dtype: int64

In [13]:
df[(df.classification == 1)].count()

# Similar numbers of 0s and 1s

title             21417
text              21417
subject           21417
date              21417
classification    21417
dtype: int64

In [14]:
df.count()
# Total number of entries

title             44898
text              44898
subject           44898
date              44898
classification    44898
dtype: int64

In [15]:
df.iloc[0,0]

' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing'

In [16]:
df['text'] = df['text'] + " " + df['title']
del df['title']
del df['subject']
del df['date']

# Only using text and classification columns

In [17]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)
# Removing stop words

In [18]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['text']=df['text'].apply(denoise_text)


# https://www.kaggle.com/madz2000/nlp-using-glove-embeddings-99-87-accuracy

In [19]:
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(df.text)
corpus[:5]

['Donald', 'Trump', 'wish', 'Americans', 'Happy']

In [20]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = dict(most_common)
most_common

{'Trump': 111495,
 'said': 93159,
 'would': 54611,
 'U.S.': 50343,
 'people': 33110,
 'President': 33093,
 'also': 30325,
 'one': 29369,
 'Donald': 27708,
 'said.': 26194}

#### Preparations of train and test datasets

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.classification, test_size = 0.3, random_state=0)

In [22]:
X_train2, X_test2, y_train2, y_test2 = X_train, X_test, y_train, y_test


In [23]:
tokenizer = text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(tokenized_train, maxlen=300)

tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=300)

# Tokenizing from string to floats

### MLP Classifier

In [25]:
clf = MLPClassifier(random_state=0, max_iter=300).fit(X_train, y_train)

In [26]:
clf_predict = clf.predict(X_test)

In [27]:
print('Accuracy is {:.3f}'.format(np.mean(clf_predict == y_test)))
print('f1-Score is {:.3f}'.format(f1_score(y_test, clf_predict)))

Accuracy is 0.621
f1-Score is 0.435


In [28]:
print(classification_report(y_test, clf_predict, target_names = ['Fake','Not Fake']))

              precision    recall  f1-score   support

        Fake       0.59      0.91      0.72      7025
    Not Fake       0.76      0.31      0.44      6445

    accuracy                           0.62     13470
   macro avg       0.67      0.61      0.58     13470
weighted avg       0.67      0.62      0.58     13470



In [29]:
tn, fp, fn, tp = confusion_matrix(y_test, clf_predict).ravel()

In [30]:
print(tp/(tp+(0.5*(fp+fn))))

0.435494578446559


### Naive Bayes classification

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train2)
X_train_counts.shape

(31428, 103262)

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(31428, 103262)

In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train2)

In [35]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

In [36]:
text_clf = text_clf.fit(X_train2, y_train2)

In [37]:
predicted = text_clf.predict(X_test2)
print('Accuracy is {:.3f}'.format(np.mean(predicted == y_test2)))

Accuracy is 0.938


In [38]:
print('Accuracy is {:.3f}'.format(np.mean(predicted == y_test2)))
print('f1-Score is {:.3f}'.format(f1_score(y_test2, predicted)))

Accuracy is 0.938
f1-Score is 0.935


In [39]:
print(classification_report(y_test2, predicted, target_names = ['Fake','Not Fake']))

              precision    recall  f1-score   support

        Fake       0.93      0.95      0.94      7025
    Not Fake       0.95      0.92      0.93      6445

    accuracy                           0.94     13470
   macro avg       0.94      0.94      0.94     13470
weighted avg       0.94      0.94      0.94     13470



### SGD Classifier

In [40]:

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, random_state=0))])
_ = text_clf_svm.fit(X_train2, y_train2)


In [41]:
predicted_svm = text_clf_svm.predict(X_test2)

In [42]:
print('Accuracy is {:.3f}'.format(np.mean(predicted_svm == y_test2)))
print('f1-Score is {:.3f}'.format(f1_score(y_test2, predicted_svm)))

Accuracy is 0.974
f1-Score is 0.973


In [43]:
print(classification_report(y_test2, predicted_svm, target_names = ['Fake','Not Fake']))

              precision    recall  f1-score   support

        Fake       0.98      0.97      0.97      7025
    Not Fake       0.97      0.98      0.97      6445

    accuracy                           0.97     13470
   macro avg       0.97      0.97      0.97     13470
weighted avg       0.97      0.97      0.97     13470



## Summary


| Algorithms | Accuracy | f1-Score |
| --- | --- | --- |
| <b>Naive Bayes</b> | 0.938 | 0.935 |
| <b>MLPClassifier</b> | 0.621 | 0.435 |
| <b>SGDClassifier</b> | 0.974 | 0.973 |

<b>Naive Bayes algorithm</b> was chosen as it is a good text classification algorithm, however it does not consider the correlation between factors in the text.<br>
<b>SGDClassifier</b> was chosen as it treats the data in batches and performs gradient descent targetted to minimize expected loss with respect to the data's distribution.<br>
<b>MLPClassifier</b> was selected due to its universal approach capability and its flexibility to form quality solutions for a wide range of attributes of the same algorithm of learning.

Overall <b>SGDClassifier</b> is most preferred as it has the highest Accuracy and f1-Score. <br>

Although theoretically,<b>MLPClassifier</b> is expected to have the best performance, due to its universal approach capability. It is also been sucessfully applied to most diverse problems through Neural Networks.<br>
In order to increase the accuracy and f1-score, we can increase the <b>learning rate</b>, <b>number of epochs</b> and <b>shuffle our dataset</b>.

## Others

#### LSTM

In [44]:
embedding_file = 'C:/Users/Boonchong/Desktop/Intern/FakeNews/glove.twitter.27B.100d.txt'
# Source https://www.kaggle.com/bertcarremans/glovetwitter27b100dtxt

In [45]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embedding_file, encoding="utf8"))

In [46]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(10000, len(word_index))


embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= 10000: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3347: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [47]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [48]:
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(10000, output_dim=embed_size, weights=[embedding_matrix], input_length=300, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

# https://www.kaggle.com/madz2000/nlp-using-glove-embeddings-99-87-accuracy

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 300, 128)          117248    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________


In [50]:
history = model.fit(X_train, y_train, batch_size = 256 , validation_data = (X_test,y_test) , epochs = 10)

Epoch 1/10
 43/123 [=========>....................] - ETA: 7:14 - loss: 0.4978 - accuracy: 0.7749

KeyboardInterrupt: 

In [51]:
pred = model.predict_classes(X_test)

C:\Users\Boonchong\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [52]:
print(classification_report(y_test, pred, target_names = ['Fake','Not Fake']))

              precision    recall  f1-score   support

        Fake       0.98      0.96      0.97      7025
    Not Fake       0.96      0.98      0.97      6445

    accuracy                           0.97     13470
   macro avg       0.97      0.97      0.97     13470
weighted avg       0.97      0.97      0.97     13470

